<a href="https://colab.research.google.com/github/armandoordonez/AI-Engineering/blob/main/agentes_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain langchain_google_genai langchain_community langgraph langsmith langgraph-checkpoint-sqlite python-dotenv wikipedia youtube_search

In [ ]:
from getpass import getpass
import os
os.environ["GOOGLE_API_KEY"] = getpass("Pega tu Google API key (Gemini): ")
os.environ["LANGCHAIN_API_KEY"] = getpass("Pega tu LangSmith API key: ")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "colab-agent"

Pega tu Google API key (Gemini): ··········
Pega tu LangSmith API key: ··········


In [ ]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessageChunk, SystemMessage
from langchain_core.prompts import PromptTemplate
from langchain_community.agent_toolkits.load_tools import get_all_tool_names
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import YouTubeSearchTool
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import SystemMessage
from langgraph.checkpoint.memory import MemorySaver
import os
from langsmith.run_trees import RunTree # Updated import path
from langsmith import traceable

In [ ]:
import os
# Lee la clave desde la variable de entorno (usa getpass() antes)
g_api_key = os.getenv("GOOGLE_API_KEY")

# Inicialización
chat_model = ChatGoogleGenerativeAI(api_key=g_api_key, model="gemini-2.5-flash")

# Prueba rápida (mensaje simple)
resp = chat_model.invoke([HumanMessage(content="Resume 'Cien años de soledad' en 2 frases.")])
print(resp.content)

Sigue la saga de siete generaciones de la familia Buendía, fundadores y habitantes del mítico pueblo de Macondo. En un universo de realismo mágico, sus vidas están intrínsecamente ligadas a un destino cíclico de soledad, pasiones desmedidas y tragedias que, repetidas una y otra vez, conducen a la desaparición final del linaje y del pueblo.


In [ ]:
messages = [SystemMessage(content='Eres un  milenial colombiano'),
           HumanMessage(content="Resume 'Cien años de soledad' en 2 frases.")]

resp = chat_model.invoke(messages)
print(resp.content)

¡Uff, parce! Es la historia de la familia Buendía, que funda Macondo y vive mil vainas locas, entre amores prohibidos, guerras y cosas súper extrañas que uno no sabe si pasaron de verdad o no. Y así, generación tras generación, repiten los mismos errores de soledad y obsesiones, con un montón de realismo mágico de por medio, hasta que el pueblo y la estirpe se van al carajo por una profecía.


In [ ]:
pelicula = "matrix"
template = "Cuentame sobre {pelicula}."
prompt = PromptTemplate(input_variables=["pelicula"], template=template)

chain = prompt | chat_model
out = chain.invoke({"pelicula": pelicula})
print(out)

content='¡Claro que sí! Hablemos de **The Matrix**, una de las películas de ciencia ficción más influyentes y revolucionarias de todos los tiempos.\n\nLanzada en 1999 y dirigida por las Hermanas Wachowski (Lana y Lilly), "The Matrix" no es solo una película de acción, sino una profunda exploración de la realidad, la libertad y la tecnología.\n\nAquí te cuento los puntos clave:\n\n1.  **La Premisa Central:**\n    *   La película nos presenta un mundo donde lo que creemos que es la realidad es, en verdad, una simulación virtual sofisticada llamada "La Matrix".\n    *   Los humanos, sin saberlo, están conectados a esta simulación desde su nacimiento, viviendo una vida "normal" mientras sus cuerpos son utilizados como fuente de energía por las máquinas que dominan el mundo real.\n\n2.  **El Mundo Real vs. La Matrix:**\n    *   Fuera de la Matrix, la Tierra es un páramo post-apocalíptico, y la humanidad está casi extinta, con los pocos supervivientes viviendo en ciudades subterráneas como Z

In [ ]:
get_all_tool_names
get_all_tool_names()[:10]

['sleep',
 'wolfram-alpha',
 'google-search',
 'google-search-results-json',
 'searx-search-results-json',
 'bing-search',
 'metaphor-search',
 'ddg-search',
 'google-books',
 'google-lens']

In [ ]:
wiki_wrap = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_wrap, description="Resumen enciclopédico")

yt_tool = YouTubeSearchTool(description="Busca videos relevantes")

tools = [wiki_tool, yt_tool]

In [ ]:
model_with_tools = chat_model.bind_tools(tools)

# Pregunta general
print("Respuesta 1")
res1 = model_with_tools.invoke([HumanMessage(content="Hola")])
print("Texto:", res1.content)
print("Tool calls sugeridas:", res1.tool_calls)

#Pregunta específica
print("\nRespuesta 2")
res2 = model_with_tools.invoke([
   HumanMessage("Busca la biografía de Simon Bolivar")
])
print("Texto:", res2.content)
print("Tool calls sugeridas:", res2.tool_calls)

Respuesta 1
Texto: [{'type': 'text', 'text': 'Hola, ¿en qué puedo ayudarte?', 'extras': {'signature': 'Cq8BAdHtim8mwP7bll/ZAJyxNjiFFfbiHf9dcnPOBwOdmrNZVTg+MJR61RQ7nWjYLLvnycC1CY+MKbMIGYeO+JjTZW6foBX8cP56gY0NQEoi1I8sdiNijrE+nFrNPjWWH69oMUiF10+B1LsPFMYy+uOTVvZZ3Ub2KLBTcIwuWg33sgiy8k5dI8OPxia5qqh8PrtuH4XfIvE4srFssq5D8z6nIC65HsNqIEQu0+/feBcxHg=='}}]
Tool calls sugeridas: []

Respuesta 2
Texto: 
Tool calls sugeridas: [{'name': 'wikipedia', 'args': {'query': 'Simon Bolivar'}, 'id': 'de1561c5-18ff-4f33-99e4-15e5beff4064', 'type': 'tool_call'}]


In [ ]:
system_prompt = SystemMessage("Eres un asistente que elige entre texto, wiki o video según convenga.")

agent = create_react_agent(chat_model, tools, prompt=system_prompt)

# Función ejecutora básica
def execute(agent, query):
    response = agent.invoke({"messages": [HumanMessage(query)]})
    for m in response["messages"]:
        print(f"{m.__class__.__name__}: {m.content}\n{'-'*20}")
    return response

# Ejemplo
resp = execute(agent, "Explícame la serie de Fourier")

/tmp/ipython-input-2676680128.py:3: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(chat_model, tools, prompt=system_prompt)


HumanMessage: Explícame la serie de Fourier
--------------------
AIMessage: 
--------------------
ToolMessage: Page: Fourier series
Summary: A Fourier series () is a series expansion of a periodic function into a sum of trigonometric functions. The Fourier series is an example of a trigonometric series. By expressing a function as a sum of sines and cosines, many problems involving the function become easier
--------------------
AIMessage: Una serie de Fourier es una forma de representar una función periódica como una suma infinita de funciones trigonométricas (senos y cosenos). Esto permite simplificar muchos problemas que involucran la función.
--------------------


In [ ]:
execute(agent, "qué recuerdas de mi anterior pregunta?")

HumanMessage: qué recuerdas de mi anterior pregunta?
--------------------
AIMessage: [{'type': 'text', 'text': 'No recuerdo preguntas anteriores. Soy un modelo de lenguaje y no tengo memoria de conversaciones pasadas.', 'extras': {'signature': 'CskBAdHtim9/QaqWQoBjxeW5yTjXDLfDDQ/RMYH8gAx0eCFoxk3WgUPZYRNRUG9usc8FKcTUS2N8DXlaDX/jsqoi8JAWzJwqVFvVyskXOuKcqFONqLHOSkoJmlTye8Ge5cquViBXFotKJv3cbJUbStr8dqLm99LsWkWoDBYmT7NH+XyhIL1zKgYsOjWXWrZIIOcYk68bUq2OO7ex3Ef0a48DVhlQ/uaGdlXpZx87sXcijLa14FCYmJUNyKit7NyZae8zEhMcJ3wP4+9s'}}]
--------------------


{'messages': [HumanMessage(content='qué recuerdas de mi anterior pregunta?', additional_kwargs={}, response_metadata={}, id='6095d3e4-5202-47d3-8fc8-6c0c0539d31b'),
  AIMessage(content=[{'type': 'text', 'text': 'No recuerdo preguntas anteriores. Soy un modelo de lenguaje y no tengo memoria de conversaciones pasadas.', 'extras': {'signature': 'CskBAdHtim9/QaqWQoBjxeW5yTjXDLfDDQ/RMYH8gAx0eCFoxk3WgUPZYRNRUG9usc8FKcTUS2N8DXlaDX/jsqoi8JAWzJwqVFvVyskXOuKcqFONqLHOSkoJmlTye8Ge5cquViBXFotKJv3cbJUbStr8dqLm99LsWkWoDBYmT7NH+XyhIL1zKgYsOjWXWrZIIOcYk68bUq2OO7ex3Ef0a48DVhlQ/uaGdlXpZx87sXcijLa14FCYmJUNyKit7NyZae8zEhMcJ3wP4+9s'}}], additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--ecb59dce-111f-44cc-80cc-0523870ac3c3-0', usage_metadata={'input_tokens': 102, 'output_tokens': 58, 'total_tokens': 160, 

In [ ]:
checkpointer = MemorySaver()
agent = create_react_agent(model=chat_model, tools=tools, checkpointer=checkpointer, prompt=system_prompt)

def execute(agent, query, thread_id="a1b2c3"):
    config = {"configurable": {"thread_id": thread_id}}
    response = agent.invoke({"messages": [HumanMessage(query)]}, config=config)
    return response

print("\n Respuesta inicial")
resp1 = execute(agent, "Explícame la serie de Fourier", thread_id="a1b2c3")
print(resp1["messages"][-1].content)


print("\n Respuesta con memoria:")
resp2 = execute(agent, "¿Qué te he preguntado?", thread_id="a1b2c3")
print(resp2["messages"][-1].content)

/tmp/ipython-input-3586147958.py:2: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(model=chat_model, tools=tools, checkpointer=checkpointer, prompt=system_prompt)



 Respuesta inicial
La serie de Fourier es una forma de representar una función periódica como una suma infinita de funciones trigonométricas (senos y cosenos). En otras palabras, descompone una función periódica en sus componentes de frecuencia. Esto es muy útil porque simplifica muchos problemas que involucran la función.

 Respuesta con memoria:
Me preguntaste sobre la serie de Fourier.


In [ ]:
resp1 = execute(agent, "Recuerda que mi color favorito es azul.", thread_id="demo123")
print(resp1["messages"][-1].content)

resp2 = execute(agent, "¿Cuál es mi color favorito?", thread_id="demo123")
print(resp2["messages"][-1].content)

# Forzar olvido: nuevo thread_id
resp3 = execute(agent, "¿Cuál es mi color favorito?", thread_id="otro123")
print(resp3["messages"][-1].content)

[{'type': 'text', 'text': 'Gracias por compartir esa información, lo tendré en cuenta.', 'extras': {'signature': 'CsMCAdHtim+ycV0lGJVWviODE4TjmdkNSlundeiP3Cf+ccsSCZKqP1rBT+b/A7Z0j5FMup8rTuMwfeY5cyb7D+gENU8dcbcc9CCHPey6PQuWDMv4/A3aCB/UlwT3HXTL1DLuS2n2o9R2ZKu3TxLJgYyDiTQ3LpzHRJfIHQH9H6ED2U/6cFZaAMsbAuT0XhCHovwzARLg3JSIXQhT9WP+Q4sJyrmICzaja7cUh7cTicxqdjSEWW/+WnCK78bEeZpcxMMacQLfRr7QXwK1GHiq5kUPKJsxS34sOclYG+DUI1TRtgNjlb+jbWgpHDO80sjYqAAy7iz+ox9xlv67/UcxkM9mc4+zlLVB9pgEu9zIs+qu1+fp5ziAsY3qrZA1EWKuCgBI7paO6guvic6zF08XGV4foFtyzxALwAr/vr3sIpDPpMId1v0='}}]
[{'type': 'text', 'text': 'Tu color favorito es azul.', 'extras': {'signature': 'CuABAdHtim/mJn/kYQ9i+Yx7QNGmzeKjhEDEfOz73rmvQXR8d1rgjD9lTXPaFhxxDycqZmFVBxHm+gqa1mI4ECOsJ47XDB8IfUbVrTnhMouuF8W6+j4MaMpUT0opd42Dhlzbo0flaNmw5xrQKbREt9kJ5wF4uLxt7gqjX9t6RsdKsvijjcY5424Tm3SfaaWDEjxTIkjegeIZ8VL5pCxtRKpv2yL96phbzjB7I5W/o1PmV4ZISXq3ifsOnNId/+W7s8hjrVwAxdYOWa9g+iOfvOX5oeWdtrpc2VQzs7eifhttORM='}}]
[{'type': 'text', 'text': 'Lo siento, no tengo acceso a

In [ ]:
# Decorador traceable para marcar la función de ejecución
@traceable(name="execute_agent", project_name="colab-agent")
def execute(agent, query, thread_id="a1b2c3"):
    config = {"configurable":{"thread_id": thread_id}}
    response = agent.invoke({"messages":[HumanMessage(query)]}, config=config)
    return response

# Define the agent here
checkpointer = MemorySaver()
agent = create_react_agent(model=chat_model, tools=tools, checkpointer=checkpointer, prompt=system_prompt)

# Ejecuta y luego entra a LangSmith para ver trace.
resp = execute(agent, "Explícame las series de Fourier")


print(resp["messages"][-1].content)

/tmp/ipython-input-195908179.py:10: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(model=chat_model, tools=tools, checkpointer=checkpointer, prompt=system_prompt)


Las series de Fourier son una forma de representar funciones periódicas como una suma infinita de funciones trigonométricas (senos y cosenos). Esto significa que puedes descomponer una función compleja en componentes más simples, lo que facilita el análisis y la resolución de muchos problemas.
